In [1]:
from Hamil_search import *
from simulation import *
import time
import numpy as np
from numba import jit

In [13]:
lambdaPen = 1024
blockNum = 2
n = 4
size = n*blockNum

Hpen_block = []
Htar_block = []
for i in range(blockNum):
    Htar_block.append((1, f'Z{2*i}*Z{2*i+1}'))
    Htar_block.append((1, f'Z{2*i}'))
    Htar_block.append((1, f'Z{2*i+1}'))
    Htar_block.append((1, f'X{2*i}'))
    Htar_block.append((1, f'X{2*i+1}'))
    Hpen_block += [(1, f'X{4*i}*X{4*i+1}'), (3, f'Z{4*i}*Z{4*i+1}'), (1, f'X{4*i+2}*X{4*i+3}'), (3, f'Z{4*i+2}*Z{4*i+3}')]
for i in range(blockNum-1):
    Htar_block.append((1, f'Z{2*i+1}*Z{2*i+2}'))
print(Htar_block)

Henc_block = EncodeTar(Htar_block, blockNum, lambdaPen)
print(Henc_block)
U = getU(n, blockNum)

P = U @ U.conj().T
Q = np.identity(P.shape[0]) - P
Henc = blocks2Mat(size, Henc_block)
Hpen = blocks2Mat(size, Hpen_block)
Htar = blocks2Mat(size//2, Htar_block)
HpenInverse = np.linalg.pinv(Hpen)

print(checkSame(P@Hpen, np.zeros(P.shape)))
print(checkSame(P@Hpen@Q, np.zeros(P.shape))) 
print(checkSame(P@Henc@P - (P@Henc@Q@HpenInverse@Q@Henc@P / lambdaPen), U@Htar@U.conj().T))
Hsim = lambdaPen * Hpen + Henc

[(1, 'Z0*Z1'), (1, 'Z0'), (1, 'Z1'), (1, 'X0'), (1, 'X1'), (1, 'Z2*Z3'), (1, 'Z2'), (1, 'Z3'), (1, 'X2'), (1, 'X3'), (1, 'Z1*Z2')]
[(1, 'Z1*Z2'), (1, 'Z0*Z1'), (1, 'Z0*Z2'), (-1, 'X0*X2'), (1, 'X0*X1'), (1, 'Z5*Z6'), (1, 'Z4*Z5'), (1, 'Z4*Z6'), (-1, 'X4*X6'), (1, 'X4*X5'), (1, 'Z4*Z5'), (52.255781179374466, 'Z1*X6+Z3*X6')]
True
True
True


In [3]:
np.random.seed(42)
epsilons = np.random.uniform(-0.1,0.1,size*3)
# epsilons = np.zeros(size*3)
V = [PauliTerm(size, f'X{i}', epsilons[i]) for i in range(size)] + [PauliTerm(size, f'Z{i}', epsilons[i+size]) for i in range(size)] + [PauliTerm(size, f'Y{i}', epsilons[i+size]) for i in range(size)]
V = sum([p.value() for p in V])

print(checkSame(P@V@P, np.zeros(P.shape)))

True


In [4]:
# ---- previous version ----
t = 0.1
Hleft = expm(-1j * t * (Hsim + V))
Hright = U @ expm(-1j * t * Htar) @ U.conj().T
err = np.linalg.norm((Hleft - Hright)@P, ord = 2)
print(f"t={t}, err={err}")

t=0.1, err=0.03539280396075681


In [5]:
# @jit(nopython=True)
# def exp_minus_iHt(H, t):
#     e, u = np.linalg.eigh(H)
#     e = np.expand_dims(e, axis=1)
#     A = u.conj().T
#     A = np.exp(-1j*e*t) * A
#     A = u @ A
#     return A

# @jit(nopython=True)
# def exp_minus_iHt_mult_B(H, t, B):
#     e, u = np.linalg.eigh(H)
#     e = np.expand_dims(e, axis=1)
#     A = u.conj().T @ B
#     A = np.exp(-1j*e*t) * A
#     A = u @ A
#     return A

# @jit(nopython=True)
# def time_evol_err(t, Hsim, V, Htar, U):
#     Hleft = exp_minus_iHt_mult_B(Hsim + V, t, U)
#     Hright = U @ exp_minus_iHt(Htar, t)
#     err = np.linalg.norm(Hleft - Hright, ord = 2)
#     return err

In [6]:
def exp_minus_iHt(H, t):
    """
    e^{-iHt}
    """
    e, u = np.linalg.eigh(H)
    e = np.expand_dims(e, axis=1)
    A = u.conj().T
    A = np.exp(-1j*e*t) * A
    A = u @ A
    return A

def exp_minus_iHt_mult_B(H, t, B):
    """
    e^{-iHt} B
    """
    e, u = np.linalg.eigh(H)
    e = np.expand_dims(e, axis=1)
    A = u.conj().T @ B
    A = np.exp(-1j*e*t) * A
    A = u @ A
    return A

def time_evol_err(t, Hsim, V, Htar, U):
    Hleft = exp_minus_iHt_mult_B(Hsim + V, t, U)
    Hright = U @ exp_minus_iHt(Htar, t)
    err = np.linalg.norm(Hleft - Hright, ord = 2)
    return err

In [7]:
time_evol_err(0.1, Hsim, V, Htar, U)

0.03539280396075802

In [8]:
t = 0.1
Hleft = expm(-1j * t * (Hsim + V)) @ U
Hright = U @ expm(-1j * t * Htar)
np.linalg.norm(Hleft - Hright, ord = 2)

0.035392803960756836

In [9]:
Q0 = HpenInverse @ Hpen
P0 = np.identity(Q0.shape[0]) - Q0
Heff_2nd_order = P0@Henc@P0 - (P0@Henc@Q0@HpenInverse@Q0@Henc@P0 / lambdaPen)

In [10]:
print(checkSame(P @ Heff_2nd_order @ P, U@Htar@U.conj().T))

True


In [11]:
print(checkSame(P @ Heff_2nd_order @ (P0-P), np.zeros(P.shape)))

True
